In [1]:
import build_dataset as bd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/home/nclabterm1/refresh/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("[DEBUG] build_dataset:", bd.__file__)
# print([k for k in sys.modules if k.endswith("build_dataset")])


[DEBUG] build_dataset: /home/nclabterm1/refresh/train/build_dataset.py


In [3]:
import trl

print(trl.__version__)

0.21.0


In [11]:
use_datasets_list = [
    ["../dataset/my_korean", 900], 
    # ["../dataset/my_race_middle", 5000], 
    # ["../dataset/my_race_high", 2000], 
    # ["../dataset/my_cloth", 2100]
    ]

train_dataset = bd.concat_datasets(use_datasets_list, "train")
# train_dataset = bd.build_dataset("../dataset/my_korean", "train", 12)

Map: 2 examples [00:00, 291.88 examples/s]       


Map: 1800 examples [00:00, 10590.13 examples/s]      

ad


In [24]:
train_dataset.cleanup_cache_files()
del train_dataset

NameError: name 'train_dataset' is not defined

In [21]:
print(train_dataset[12]["messages"])

[{'content': "You are a helpful AI assistant.\nPlease answer the user's questions correctly.\nLook for the evidence in the text when answering.\nUnderlined replaced with highlights. Example: **Was Underline**\nReturn only the letter corresponding to the best answer.", 'role': 'system'}, {'content': '\n[QUESTION]\n글의 흐름으로 보아, 주어진 문장이 들어가기에 가장 적절한 곳을 고르시오.\nBut neither are aware of this fact when it comes to unmarked crosswalks.\n\n[PASSAGE]\nStudies do show that motorists are more likely to yield\nto pedestrians in marked crosswalks than at unmarked\ncrosswalks. But as some researchers found, that does not\nnecessarily make things safer. ( ① ) When they compared\nthe way pedestrians crossed at both kinds of crosswalks\non roads with considerable traffic volumes, they found that\npeople at unmarked crosswalks tended to look both ways\nmore often, waited more often for gaps in traffic, and\ncrossed the road more quickly. ( ② ) Researchers suspect\nthat both drivers and pedestrians are mor

In [10]:
# 1) 데이터셋에서 샘플 하나 꺼내기
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
sample = train_dataset[2]
print("==== raw sample ====")
print(sample)                 # messages 리스트가 제대로 들어 있는지 우선 확인

# 2) 템플릿 적용 → 사람이 읽을 수 있는 문자열로
prompt_str = tokenizer.apply_chat_template(
    sample["messages"],
    tokenize=False,           # => 문자열
    add_generation_prompt=False
)
print("\n==== prompt string ====")
print(prompt_str)

# 3) EOS/BOS 개수 세어 보기
eos_tok = tokenizer.eos_token or "<|eos|>"
bos_tok = tokenizer.bos_token or "<bos>"

print("\nEOS count:", prompt_str.count(eos_tok))
print("BOS count:", prompt_str.count(bos_tok))


==== raw sample ====
{'messages': [{'content': "You are a helpful AI assistant.\nPlease answer the user's questions correctly.\nLook for the evidence in the text when answering.\nUnderlined replaced with highlights. Example: **Was Underline**\nReturn only the letter corresponding to the best answer.", 'role': 'system'}, {'content': '\n[QUESTION]\n다음 빈칸에 공통으로 들어갈 말로 가장 적절한 것을 고르시오.\n\n[PASSAGE]\n◦ She has a big smile on her _ .\n◦ You should learn to _ your problem.\n\n[OPTIONS]\nA. face\nB. heat\nC. meet\nD. walk\n\n[ANSWER]\n', 'role': 'user'}, {'content': 'A', 'role': 'assistant'}]}

==== prompt string ====
<bos><start_of_turn>user
You are a helpful AI assistant.
Please answer the user's questions correctly.
Look for the evidence in the text when answering.
Underlined replaced with highlights. Example: **Was Underline**
Return only the letter corresponding to the best answer.

[QUESTION]
다음 빈칸에 공통으로 들어갈 말로 가장 적절한 것을 고르시오.

[PASSAGE]
◦ She has a big smile on her _ .
◦ You should learn

In [6]:

option_letters = ["A", "B", "C", "D", "E"]
option_token_id = []

for letter in option_letters:
    id = tokenizer.encode(letter, add_special_tokens=False)
    option_token_id.append(id[0])

print(option_token_id)

[236776, 236799, 236780, 236796, 236788]
